In [13]:
import torchvision
torchvision.__version__

'0.10.1+cpu'

In [14]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
root='data',
train=True,
transform=ToTensor(),
download=True,
)
test_data=datasets.MNIST(
root='data',
train=False,
transform=ToTensor()
)

In [15]:
loaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1),
    'test': torch.utils.data.DataLoader(test_data,batch_size=100, shuffle=True, num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1f1e0d00910>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x1f1e0d007c0>}

In [16]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        x = x.view(x.size(0), -1)
        output = self.out(x)

        return output, x

In [17]:
cnn=CNN()
loss_func = nn.CrossEntropyLoss()

In [18]:
from torch import optim

optimizer=optim.Adam(cnn.parameters(), lr=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [19]:
from torch.autograd import Variable
num_epochs=10

def train(num_epochs, cnn, loaders):
    cnn.train()

    #train the models
    total_step=len(loaders['train'])

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            b_x = Variable(images)
            b_y = Variable(labels)
            output = cnn(b_x)[0]
            loss = loss_func(output, b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print('Epoch [{}/{}],Step[{}/{}],Loss:{:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                pass
            pass
        pass


In [20]:
train(num_epochs, cnn, loaders)

Epoch [1/10],Step[100/600],Loss:0.2781
Epoch [1/10],Step[200/600],Loss:0.1403
Epoch [1/10],Step[300/600],Loss:0.2220
Epoch [1/10],Step[400/600],Loss:0.1225
Epoch [1/10],Step[500/600],Loss:0.0115
Epoch [1/10],Step[600/600],Loss:0.0196
Epoch [2/10],Step[100/600],Loss:0.0728
Epoch [2/10],Step[200/600],Loss:0.0263
Epoch [2/10],Step[300/600],Loss:0.0648
Epoch [2/10],Step[400/600],Loss:0.0826
Epoch [2/10],Step[500/600],Loss:0.1218
Epoch [2/10],Step[600/600],Loss:0.0235
Epoch [3/10],Step[100/600],Loss:0.0049
Epoch [3/10],Step[200/600],Loss:0.0499
Epoch [3/10],Step[300/600],Loss:0.0210
Epoch [3/10],Step[400/600],Loss:0.0148
Epoch [3/10],Step[500/600],Loss:0.0038
Epoch [3/10],Step[600/600],Loss:0.0430
Epoch [4/10],Step[100/600],Loss:0.0549
Epoch [4/10],Step[200/600],Loss:0.0325
Epoch [4/10],Step[300/600],Loss:0.0821
Epoch [4/10],Step[400/600],Loss:0.0487
Epoch [4/10],Step[500/600],Loss:0.0356
Epoch [4/10],Step[600/600],Loss:0.0493
Epoch [5/10],Step[100/600],Loss:0.0098
Epoch [5/10],Step[200/600

In [26]:
def test():
    cnn.eval()
    with torch.no_grad():
        correct=0
        total=0
        for images,labels in loaders['test']:
            test_output, last_layer=cnn(images)
            pred_y=torch.max(test_output,1)[1].data.squeeze()
            accuracy=(pred_y==labels).sum().item()/ float(labels.size(0))
            pass
        print('Test Accuracy: ' + str(accuracy))

In [27]:
test()


Test Accuracy: 0.97
